In [59]:
import pandas as pd
import numpy as np
import gensim
from nltk import RegexpTokenizer
from nltk.corpus import stopwords

In [162]:
data = pd.read_pickle('data_clean_4cols.pickle')
candidate_data = pd.DataFrame()
candidate_data = (data
                  .drop(['id', 'message'], axis=1)
                  .drop_duplicates('from_name')
                  .set_index('from_name'))

In [163]:
G = data.groupby('from_name',as_index=False)['message'].apply(' '.join)
candidate_data['Partei_ABK']=G.values
candidate_data.columns = ['text']

In [164]:
tokenizer = RegexpTokenizer(r'\w+')
stopword_set = set(stopwords.words('german'))
#This function does all cleaning of data using two objects above
def nlp_clean(data):
    new_data = []
    for d in data:
        new_str = d.lower()
        dlist = tokenizer.tokenize(new_str)
        dlist = list(set(dlist).difference(stopword_set))
        new_data.append(dlist)
    return new_data

In [166]:
candidate_data['text'] = candidate_data['text'].str.replace('\d+', '')#deleting nums

In [167]:
candidate_data['text']= nlp_clean(candidate_data['text'])

In [168]:
candidate_data #some candidates have party names next to their names

,text
from_name,
Valentin Abel,"[engagierten, volkshaus, direktkandidaten, ver..."
Dr. Michael von Abercron,"[verhöhnt, interessant, rnk, osten, gesicht, g..."
Grigorios Aggelidis,"[kraft, engagierten, reparieren, brügermeister..."
Diyar Agu,"[form, courage, kraft, überfluss, arbeitsplätz..."
Gökay Akbulut DIE LINKE,"[löst, interessant, rauch, hinsicht, regime, r..."
Rolf Albach,"[hervorgeholt, anschauen, mdep, bildungszentru..."
Stephan Albani MdB,"[courage, systeme, anschauen, regen, lautete, ..."
Katrin Albsteiger,"[kamsdorf, aufgefordert, kraft, gesponsert, se..."
Daniel Alff,"[form, wahres, löst, einwirkung, kraft, arbeit..."


In [169]:
#ok, so now we need to prepare an object for word2vec
class LabeledLineSentence(object):
    def __init__(self, doc_list, labels_list):
        self.labels_list = list(labels_list)
        self.doc_list = doc_list
    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
              yield gensim.models.doc2vec.LabeledSentence(doc, [self.labels_list[idx]])

In [173]:
ourobject = LabeledLineSentence(candidate_data['text'], candidate_data.index)

In [177]:
model = gensim.models.Doc2Vec(size=300, min_count=0, alpha=0.025, min_alpha=0.025)
model.build_vocab(ourobject)
#training of model
for epoch in range(2):
    model.train(ourobject, total_examples=model.corpus_count, epochs=model.iter)
    model.alpha -= 0.002
    model.min_alpha = model.alpha
#saving the created model
model.save('doc2vec.model')

In [12]:
model = gensim.models.Doc2Vec(size=300, min_count=0, alpha=0.025, min_alpha=0.025)
model.build_vocab(labeled_docs)
#training of model
for epoch in range(2):
    model.train(labeled_docs, total_examples=model.corpus_count, epochs=model.iter)
    model.alpha -= 0.002
    model.min_alpha = model.alpha
    model.train(labeled_docs, total_examples=model.corpus_count, epochs=model.iter)
#saving the created model
model.save('doc2vec.model')

2017-12-25 16:02:52 WARNING Each 'words' should be a list of words (usually unicode strings). First 'words' here is instead plain <class 'str'>.


In [20]:
def tsne(model):
    doc_2d = []

    for doc in model.docvecs:
        doc_2d.append(TSNE().fit_transform(doc).tolist()[0])

    return np.asarray(doc_2d)   # return ndarray


def plot_model(doc_2d, labels):
    plt.figure(num=1, figsize=(80, 80), facecolor="w", edgecolor="k")

    for label, doc in zip(labels, doc_2d):
        plt.plot(doc[0], doc[1], ".")
        plt.annotate(label, (doc[0], doc[1]))
  

KeyError: "word 'gerade' not in vocabulary"

In [54]:
candidate_data[:1]

,Partei_ABK,messages
from_name,,
Valentin Abel,FDP,Gerade einmal 9.000 Anträge auf die Kaufprämie...
